# 4.2 Definitions: Bits 💡, surprisal 😱 and entropy 🧐
### *Applications in language modeling (LM)*

**Hanna Hubarava** 😎 & **Alison Y. Kim** 🤩<br>
Computational Psycholinguistics<br>
University of Zurich<br>
13. March 2023<br>

### 💡 **Measuring information: bits**
* TL;DR: in information theory, a **bit** is a unit of information
* Read more here: <a href="https://eng.libretexts.org/Bookshelves/Computer_Science/Operating_Systems/Book%3A_Think_OS_-_A_Brief_Introduction_to_Operating_Systems_(Downey)/03%3A_Virtual_memory/3.01%3A_A_bit_of_information_theory">3.1: A bit of information theory</a>

### 😱 **Surprisal** (a.k.a. *information content*)

* Plain English: measures the amount of information gained when an event occurs which had some probability value associated with it
* Mathematically: for some instance or outcome $ x_i $ of random variable $ X $, which can take on values $ x_1, x_2, ... $, and the probability of outcome $ x_{i} $, $ p(x_{i}) $, the surprisal of $ x_{i} $ is given by $$ h(x_{i}) = -\log_{2}{p(x_{i})} \text{ bits} $$
* $ p(x_{i}) = 1 \Rightarrow h(x_{i}) = 0 \text{ bits} $
* $ p(x_{i}) = 0 \Rightarrow h(x_{i}) = \infty \text{ bits} $

### 🧐 **Shannon entropy**
* Plain English: measures the uncertainty of a random event $ X $, with possible outcomes $ x_{1}, x_{2}, \dots $ and associated probabilities $ P(x_{1}), P(x_{2}), \dots $
* Mathematically: the entropy of a random event $ X $, with possible outcomes $ x_{1}, x_{2}, \dots $ and distributed according to $ P : X \rightarrow [0, 1] $, is given by $$ H(X) = -\sum\limits_{x \in X} {P(x) \log_2{P(x)}} \text{ bits} $$
* The surprisal of each outcome is weighted by its probability
* Thus, one can think of Shannon entropy as the <strong>average</strong> information content
* Note: in the event that $ P(x) = 0 $, the summand $ P(x) \log_{2}{P(x)} = 0 \log_2{0} $ is taken to be $ 0 $

### **Questions 4-2: Understanding entropy**
##### 1. *Does the entropy of a random variable depend on the number of different values that the variable can take? Explain your answer using the formula that defines entropy.*
**No.** Entropy depends on neither the outcomes themselves nor how many outcomes there are. One can see that the summand depends exclusively on the probability associated with the outcome: there is no term in the summand that refers to the number of outcomes.

Consider two random variables that each have 2 possible outcomes: $ X $ which is uniformly distributed, i.e. $ P(x_{1}) = P(x_{2}) = 0.5 $, and $ Y $ which is distributed such that $ P(y_{1}) = 0.1, P(y_{2}) = 0.9 $. The entropies of $ X $ and $ Y $ are given by, respectively, $$ H(X) = -[0.5 \times \log_{2}(0.5) + 0.5 \times \log_{2}(0.5)] = 1 $$ $$ H(Y) = -[0.1 \times \log_{2}(0.1) + 0.9 \times \log_{2}(0.9)] \approx 0.469 $$
Even though $ X $ and $ Y $ have the same number of possible outcomes, they differ in entropy because the distributions of their respective outcomes differ.

##### 2. *Does the entropy of a random variable depend on the distribution of the different values that the variable can take?*
**Yes.** Entropy is a function of the distribution of the values or outcomes associated with variable $ X $, i.e. the respective probabilities associated with outcomes $ x_{1}, x_{2}, \dots $.

##### 3. *Is the <s>random variable</s> entropy of a uniformly distributed random variable high or low?*
**High — in fact, it is maximal for the given random variable.** We can think about this qualitatively. Entropy is a measure of uncertainty or lack of information. The distribution that gives us the *least* amount of information is the one with the *highest* amount of uncertainty. If a random variable $ X $, e.g. an utterance, can take on any of its possible values with equal probability as the other possible values, then the random variable has high uncertainty. Conversely, a more informative distribution (e.g. one that is highly peaked around a particular outcome) reduces the amount of uncertainty in comparison with a uniform distribution.

##### 5. *What is the difference between entropy and surprisal?*
For a random variable $ X $:
* **Surprisal** is the amount of information learned from one instance of $ X $ with outcome $ x_i $: $ h(x_{i}) = \log_{2}{p(x_{i})} $. Think of emoji: 😱.
* **Entropy** is the uncertainty of $ X $, which can take on values $ x_{1}, x_{2}, \dots $. It is the expected or average surprisal: $ H(X) = -\sum\limits_{x \in X} {P(x) \log_{2}{P(x)}} \text{ bits} $. Think of emoji: 🧐.


##### 6. *Give a linguistic example to illustrate the difference between entropy and surprisal.*
Our example is based on probabilistic LM 🔠📈💻.

In [ ]:
# Define toy vocabulary: key = NL token, value: vocabulary index
model_vocab = {'<BOS>': 0, '<EOS>': 1, 'oat': 2, 'oats': 3, 'milk': 4, 'is': 5, 'are': 6, 'tasty': 7, 'yummy': 8}

# Define an utterance
sequence = '<BOS> oat milk is yummy <EOS>'
sequence = sequence.split()
print(f'Sequence: { sequence }\n')

In LM one typically cares about the probability of token $ i $ conditioned on the preceding token(s) $ 0, \dots, i-1 $. For this example, we create an artificial probability tensor of size $ [(\text{length of sequence } S) - 1] \times [\text{length of vocab } V] $.

In [ ]:
import numpy as np
import torch


np.random.seed(3)
rows = len(sequence) - 1
columns = len(model_vocab)
probs = np.random.dirichlet(
    alpha=np.ones(columns),
    size=rows
)
probs = torch.tensor(probs, dtype=torch.float32)
for i in range(len(probs)):
	assert torch.isclose(torch.sum(probs[i]), torch.tensor(1.)) # check that each row sums to 1

print(f'Sequence: { sequence }\n')
print(f'Model vocabulary items: { model_vocab.keys() }\n')
print('Probabilities of all vocabulary items at token position...')
for i in range(len(probs)):
    print(f'  { i + 1 }: { probs[i] }')

In [ ]:
token_index = 2
token = sequence[token_index] # 'milk'
vocab_item = model_vocab[token] # vocabulary item with index 4
prob = probs[token_index-1][vocab_item] # probability of 'milk' given '<BOS> oat'
surprisal = -1 * torch.log2(prob)

print(f'The surprisal or information content of token \'{ token }\' (index { token_index }) is { surprisal } bits\n')

Continuing the LM example above, let us define random events $ X_{1, \dots, 5} $ as tokens occurring at positions $ i \in [1, 5] $. Each $ X_{i} $ can take any value $ V_{j} $ for $ j \in [0, |V|-1] $ in the model vocabulary $ V $.

In [ ]:
log_probs = torch.log2(probs) # individual log_2[P(x)] for all token positions i over all model vocabulary items j
entropy = probs * log_probs # summand for each (i, j)
entropy = -1 * torch.sum(entropy, dim=1) # sum of product of probs and log_probs, multiplied by -1

print('Per token entropies')
for i in range(len(entropy)):
    print(f'  X_{ i }, token at position { i }: { entropy[i] } bits')